# Workflows and Agents

**Preamble**: There two primary patterns we can follow according to [Anthropic](https://www.anthropic.com/research/building-effective-agents) blog post.
1. **Workflow**: 
    1. Create a scaffolding of predefined code paths around LLM calls
    2. LLMs directs control flow through predefined code paths
2. **Agent**: Remove this scaffolding (LLM directs its own **actions**, responds to **feedback**). In other words, agents don't have boundaries and they can make a decision (actions) based on the user input, context, etc.

<div style="text-align: center;">
   <img src="images/worflow_agent.webp" alt="Sample Image" width="70%">
</div>

**Why Frameworks?**

- Implementing these patterns *does not* require a framework like LangGraph.
- LangGraph aims to *minimize* overhead of implementing these patterns.
- LangGraph provides supporting infrastructure underneath ``any workflow / agent``:
    - **Persistence**
        - Memory
        - Human-In-The-Loop
    - **Streaming** 
        - From any LLM call or step in workflow / agent
    - **Deployment**
        - Testing, debugging, and deploying


## LLM Setup
For the following notebooks, we use LangGraph along Anthropic API.

In [1]:
# Required Modules

import sys
import os
import logging
import platform
from datetime import date, datetime

from dotenv import load_dotenv

from langchain_anthropic import ChatAnthropic
from pydantic import BaseModel, Field


print(f"  System: {sys.platform}")
print(f"  Platform: {platform.platform()}")
print(f"  Python version: {platform.python_version()}")
print(f"  System Execution (Python) path: {'/'.join(sys.executable.strip('/').split('/')[-3:])}")
print(f"  Last update: {date.today().strftime('%Y-%m-%d')}")

  System: darwin
  Platform: macOS-15.4.1-arm64-arm-64bit
  Python version: 3.11.12
  System Execution (Python) path: .venv/bin/python
  Last update: 2025-05-05


In [2]:
# Configure logging
logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

In [3]:
def _set_env(var: str):
    load_dotenv()  # Load variables from .env into os.environ
    if not os.environ.get(var):
        raise EnvironmentError(f"Environment variable '{var}' not found. Please set it in .env")
    else:
        logger.info(f"Environment variable '{var}' is loaded.")

# Load 
_set_env("ANTHROPIC_API_KEY")

model = "claude-3-5-sonnet-latest"
llm = ChatAnthropic(model=model)
logger.info(f"LLM model object is built wih '{model}'.")

2025-05-05 12:33:32,734 - __main__ - INFO - Environment variable 'ANTHROPIC_API_KEY' is loaded.
2025-05-05 12:33:32,736 - __main__ - INFO - LLM model object is built wih 'claude-3-5-sonnet-latest'.


# Augmented LLM
General speaking LLM's, in our context, can be used for different puposes shown in th figure.

<div style="text-align: center;">
   <img src="images/augmented_llm.webp" alt="Sample Image" width="50%">
</div>

To make sense out of it, let's go through two examples:
1. Search query
2. Tool using

## Search Query
In this example, we want to answer the user question. This is similar to what we do using, for example, chatGPT.
Note, that for building and agent we use [``Pydantic AI``](https://ai.pydantic.dev), which is a Python agent framework offering an innovative and ergonomic design.

**Note**: This is not an agent. Rather, LLM is just being used in a predefined code.

In [4]:
# Schema for structured output
class SearchQuery(BaseModel):
    search_query: str = Field(None, description="Query that is optimized web search.")
    justification: str = Field(
        None, justification="Why this query is relevant to the user's request."
    )

# Augment the LLM with schema for structured output
structured_llm = llm.with_structured_output(SearchQuery)

# Invoke the augmented LLM
output = structured_llm.invoke("How does Calcium CT score relate to high cholesterol?")
logger.info(output.search_query)
logger.info(output.justification)


2025-05-05 12:33:34,788 - httpx - INFO - HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
2025-05-05 12:33:34,798 - __main__ - INFO - relationship between Calcium CT score and high cholesterol heart disease
2025-05-05 12:33:34,799 - __main__ - INFO - Searching for information about the connection between coronary calcium scoring and cholesterol levels to understand their relationship in cardiovascular health assessment.


## Tool Using
In the second example, let's create a tool and use it to answer the user question

In [6]:
# Define a tool
def multiply(a: int, b: int) -> int:
    return a * b

# Augment the LLM with tools
llm_with_tools = llm.bind_tools([multiply])

# Invoke the LLM with input that triggers the tool call
msg = llm_with_tools.invoke("What is two times 3?")

# Get the tool call
msg.tool_calls

2025-05-05 13:17:52,264 - httpx - INFO - HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"


[{'name': 'multiply',
  'args': {'a': 2, 'b': 3},
  'id': 'toolu_01Nga8LhVy8t1FHE3M6K3Ag6',
  'type': 'tool_call'}]

As you can see, the LLM model has access to the *multiply* tool with an ID. also, it identified the arguments (in this case 2 and 3) to be used via the tool. Now, this can be used later when we will build the execution components, or pass it to the next step in the chain.

🔅 In the next notebooks, we will go through different patterns including:

- Prompt Chaining
- Prompt Paralleliztion
- Routing
- ...